In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import re
import nltk
import pandas as pd
# from urllib.parse import quote

In [13]:
df = pd.read_csv("text.csv")

In [14]:
df.head()

,Unnamed: 0,docs,category
0,0,sport pertains to any form of physical activit...,Sports
1,1,casa do pessoal do porto do lobito or simply k...,Sports
2,2,gombe united football club is a nigerian footb...,Sports
3,3,aibi international is a company operating in t...,Sports
4,4,an airboard is an inflatable bodyboard for the...,Sports


In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

unigram_vectorizer = CountVectorizer(strip_accents="ascii")
bigram_vectorizer = CountVectorizer(ngram_range=(2,2), strip_accents="ascii")
tfidf_vectorizer = TfidfVectorizer(strip_accents="ascii")

In [16]:
X = df["docs"]
y = df["category"]

In [17]:
unigram_matrix = unigram_vectorizer.fit_transform(X).toarray()
bigram_matrix = bigram_vectorizer.fit_transform(X).toarray()
tfidf_matrix = tfidf_vectorizer.fit_transform(X).toarray()

In [18]:
unigram_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]])

In [19]:
unique_words = [None for _ in range(len(unigram_vectorizer.vocabulary_))]
for word, idx in unigram_vectorizer.vocabulary_.items(): 
    unique_words[idx] = word
uni_df = pd.DataFrame(unigram_matrix, columns=unique_words)
uni_df.head()

,10,100,100m,118,12,120,13,1300,14,15,...,yet,york,you,young,younger,youth,zealand,zigzag,zoomers,zoos
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [38]:
# unique_words

In [20]:
unique_bigrams = [None for _ in range(len(bigram_vectorizer.vocabulary_))]
for word, idx in bigram_vectorizer.vocabulary_.items(): 
    unique_bigrams[idx] = word
bi_df = pd.DataFrame(bigram_matrix, columns=unique_bigrams).astype("float")
for idx, row in bi_df.iterrows():
    for bigram in unique_bigrams:
        prev = bigram.split(" ")[0]
        prev_count = uni_df[prev][idx]
        if prev_count != 0:
            row[bigram] /= prev_count
bi_df.head()

,10 and,10 climb,10 however,10 of,100 sitting,100m long,100m run,100m to,118 campus,12 swing,...,you can,young children,young men,younger children,youth and,youth through,zealand slovakia,zigzag balance,zoomers is,zoos and
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
labels = [None for _ in range(len(tfidf_vectorizer.vocabulary_))]
for word, idx in tfidf_vectorizer.vocabulary_.items(): labels[idx] = word
tfidf_df = pd.DataFrame(tfidf_matrix, columns=labels)
tfidf_df.head()

,10,100,100m,118,12,120,13,1300,14,15,...,yet,york,you,young,younger,youth,zealand,zigzag,zoomers,zoos
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019264,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.041776,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.179287,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.182381,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
import numpy as np

eps = np.finfo(float).eps

def laplace_correction(data, var, Class, target, k):
    num = k
    den = len(data[data[Class] == target]) + len(np.unique(data[var]))
    return num / (den + eps)

def naive_bayes_classifier(data, Class, instance):
    max_prob, max_prob_target = -np.inf, None
    
    targets = list(np.unique(data[Class]))
    vars = [v for v in list(data.keys()) if v != Class]

    for target in targets:
        if target ==targets[0]:
            print(data[data[Class] == target])
        p = len(data[data[Class] == target]) / (len(data) + eps)
        print("\np({}) = {:.5f}".format(target, p))
        
        for var in vars:
            num = len(data[var][data[Class] == target][data[var] == instance[var][0]])
            if num != 0:
                den = len(data[var][data[Class] == target])
                p_var = num/(den + eps)
            else:
                # Apply Laplace correction
                p_var = laplace_correction(data, var, Class, target, k=1)
            print("P({} = {} | {}) = {:.5f}".format(var, instance[var], target, p_var))
            p = p * p_var
        
        # print("P({} | {}) = {:.5f}".format(target, list(instance.values()), p))
        if p > max_prob:
            max_prob = p
            max_prob_target = target
    
    return max_prob, max_prob_target

In [41]:
new_doc = pd.Series(["sports is good for health"])

In [42]:
out = unigram_vectorizer.transform(new_doc).toarray()
uni_test = pd.DataFrame(out, columns=unigram_vectorizer.get_feature_names())
uni_test

,10,100,100m,118,12,120,13,1300,14,15,...,yet,york,you,young,younger,youth,zealand,zigzag,zoomers,zoos
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
out = bigram_vectorizer.transform(new_doc).toarray()
bi_test = pd.DataFrame(out, columns=unique_bigrams)
for bigram in unique_bigrams:
    prev = bigram.split(" ")[0]
    prev_count = uni_test[prev][0]
    if prev_count != 0:
        row[bigram] /= prev_count

In [28]:
out = tfidf_vectorizer.transform(new_doc).toarray()
tfid_test = pd.DataFrame(out, columns=tfidf_vectorizer.get_feature_names())

In [57]:
naive_bayes_classifier(uni_df.assign(category=y), "category", uni_test)


p(Education) = 0.50000
P(10 = 0    0
Name: 10, dtype: int64 | Education) = 0.80000
P(100 = 0    0
Name: 100, dtype: int64 | Education) = 0.90000
P(100m = 0    0
Name: 100m, dtype: int64 | Education) = 1.00000
P(118 = 0    0
Name: 118, dtype: int64 | Education) = 0.90000
P(12 = 0    0
Name: 12, dtype: int64 | Education) = 1.00000
P(120 = 0    0
Name: 120, dtype: int64 | Education) = 0.90000
P(13 = 0    0
Name: 13, dtype: int64 | Education) = 1.00000
P(1300 = 0    0
Name: 1300, dtype: int64 | Education) = 1.00000
P(14 = 0    0
Name: 14, dtype: int64 | Education) = 1.00000
P(15 = 0    0
Name: 15, dtype: int64 | Education) = 0.90000
P(15th = 0    0
Name: 15th, dtype: int64 | Education) = 0.90000
P(16 = 0    0
Name: 16, dtype: int64 | Education) = 1.00000
P(165 = 0    0
Name: 165, dtype: int64 | Education) = 0.90000
P(1650s = 0    0
Name: 1650s, dtype: int64 | Education) = 0.90000
P(17th = 0    0
Name: 17th, dtype: int64 | Education) = 0.90000
P(18 = 0    0
Name: 18, dtype: int64 | Educati

Name: any, dtype: int64 | Education) = 0.60000
P(anything = 0    0
Name: anything, dtype: int64 | Education) = 0.90000
P(apc = 0    0
Name: apc, dtype: int64 | Education) = 0.90000
P(application = 0    0
Name: application, dtype: int64 | Education) = 0.90000
P(applied = 0    0
Name: applied, dtype: int64 | Education) = 0.90000
P(applies = 0    0
Name: applies, dtype: int64 | Education) = 0.90000
P(apply = 0    0
Name: apply, dtype: int64 | Education) = 0.90000
P(approach = 0    0
Name: approach, dtype: int64 | Education) = 0.70000
P(approached = 0    0
Name: approached, dtype: int64 | Education) = 0.90000
P(approaches = 0    0
Name: approaches, dtype: int64 | Education) = 0.80000
P(appropriate = 0    0
Name: appropriate, dtype: int64 | Education) = 0.80000
P(aqua = 0    0
Name: aqua, dtype: int64 | Education) = 1.00000
P(aquabiking = 0    0
Name: aquabiking, dtype: int64 | Education) = 1.00000
P(aquariums = 0    0
Name: aquariums, dtype: int64 | Education) = 0.90000
P(are = 0    0
Name

Name: characterize, dtype: int64 | Education) = 0.90000
P(characterized = 0    0
Name: characterized, dtype: int64 | Education) = 0.90000
P(characterizes = 0    0
Name: characterizes, dtype: int64 | Education) = 0.90000
P(characterizing = 0    0
Name: characterizing, dtype: int64 | Education) = 0.90000
P(characters = 0    0
Name: characters, dtype: int64 | Education) = 0.90000
P(charge = 0    0
Name: charge, dtype: int64 | Education) = 0.90000
P(chartable = 0    0
Name: chartable, dtype: int64 | Education) = 1.00000
P(charter = 0    0
Name: charter, dtype: int64 | Education) = 1.00000
P(checkers = 0    0
Name: checkers, dtype: int64 | Education) = 1.00000
P(chess = 0    0
Name: chess, dtype: int64 | Education) = 1.00000
P(chicane = 0    0
Name: chicane, dtype: int64 | Education) = 1.00000
P(child = 0    0
Name: child, dtype: int64 | Education) = 0.70000
P(childhood = 0    0
Name: childhood, dtype: int64 | Education) = 0.90000
P(children = 0    0
Name: children, dtype: int64 | Education

Name: de, dtype: int64 | Education) = 1.00000
P(deadlines = 0    0
Name: deadlines, dtype: int64 | Education) = 0.90000
P(dealing = 0    0
Name: dealing, dtype: int64 | Education) = 0.90000
P(debate = 0    0
Name: debate, dtype: int64 | Education) = 0.90000
P(decade = 0    0
Name: decade, dtype: int64 | Education) = 0.90000
P(decentralized = 0    0
Name: decentralized, dtype: int64 | Education) = 0.90000
P(decet = 0    0
Name: decet, dtype: int64 | Education) = 1.00000
P(decided = 0    0
Name: decided, dtype: int64 | Education) = 1.00000
P(decisions = 0    0
Name: decisions, dtype: int64 | Education) = 0.80000
P(decline = 0    0
Name: decline, dtype: int64 | Education) = 0.90000
P(decroly = 0    0
Name: decroly, dtype: int64 | Education) = 0.90000
P(dedicated = 0    0
Name: dedicated, dtype: int64 | Education) = 0.90000
P(deep = 0    0
Name: deep, dtype: int64 | Education) = 0.90000
P(define = 0    0
Name: define, dtype: int64 | Education) = 0.70000
P(defined = 0    0
Name: defined, dt

Name: et, dtype: int64 | Education) = 0.80000
P(etc = 0    0
Name: etc, dtype: int64 | Education) = 0.90000
P(ethical = 0    0
Name: ethical, dtype: int64 | Education) = 0.90000
P(ethics = 0    0
Name: ethics, dtype: int64 | Education) = 0.80000
P(ethnicity = 0    0
Name: ethnicity, dtype: int64 | Education) = 0.90000
P(europe = 0    0
Name: europe, dtype: int64 | Education) = 0.90000
P(european = 0    0
Name: european, dtype: int64 | Education) = 0.90000
P(evaluate = 0    0
Name: evaluate, dtype: int64 | Education) = 1.00000
P(evaluated = 0    0
Name: evaluated, dtype: int64 | Education) = 0.90000
P(evaluative = 0    0
Name: evaluative, dtype: int64 | Education) = 0.80000
P(even = 0    0
Name: even, dtype: int64 | Education) = 0.80000
P(event = 0    0
Name: event, dtype: int64 | Education) = 1.00000
P(events = 0    0
Name: events, dtype: int64 | Education) = 0.60000
P(everett = 0    0
Name: everett, dtype: int64 | Education) = 0.90000
P(every = 0    0
Name: every, dtype: int64 | Educa

...

P(program = 0    0
Name: program, dtype: int64 | Sports) = 1.00000
P(programs = 0    0
Name: programs, dtype: int64 | Sports) = 1.00000
P(progress = 0    0
Name: progress, dtype: int64 | Sports) = 0.90000
P(prohibitions = 0    0
Name: prohibitions, dtype: int64 | Sports) = 1.00000
P(project = 0    0
Name: project, dtype: int64 | Sports) = 1.00000
P(prominence = 0    0
Name: prominence, dtype: int64 | Sports) = 1.00000
P(promote = 0    0
Name: promote, dtype: int64 | Sports) = 1.00000
P(promoted = 0    0
Name: promoted, dtype: int64 | Sports) = 1.00000
P(promoting = 0    0
Name: promoting, dtype: int64 | Sports) = 1.00000
P(promotion = 0    0
Name: promotion, dtype: int64 | Sports) = 0.90000
P(prompted = 0    0
Name: prompted, dtype: int64 | Sports) = 1.00000
P(proposed = 0    0
Name: proposed, dtype: int64 | Sports) = 1.00000
P(protection = 0    0
Name: protection, dtype: int64 | Sports) = 0.90000
P(proved = 0    0
Name: proved, dtype: int64 | Sports) = 0.90000
P(provide = 0    0
Name:

Name: season, dtype: int64 | Sports) = 0.80000
P(sec = 0    0
Name: sec, dtype: int64 | Sports) = 0.90000
P(second = 0    0
Name: second, dtype: int64 | Sports) = 1.00000
P(secondary = 0    0
Name: secondary, dtype: int64 | Sports) = 1.00000
P(section = 0    0
Name: section, dtype: int64 | Sports) = 0.90000
P(securing = 0    0
Name: securing, dtype: int64 | Sports) = 0.90000
P(see = 0    0
Name: see, dtype: int64 | Sports) = 0.90000
P(seek = 0    0
Name: seek, dtype: int64 | Sports) = 1.00000
P(seeks = 0    0
Name: seeks, dtype: int64 | Sports) = 1.00000
P(seen = 0    0
Name: seen, dtype: int64 | Sports) = 1.00000
P(self = 0    0
Name: self, dtype: int64 | Sports) = 1.00000
P(selling = 0    0
Name: selling, dtype: int64 | Sports) = 1.00000
P(senes = 0    0
Name: senes, dtype: int64 | Sports) = 0.90000
P(sense = 0    0
Name: sense, dtype: int64 | Sports) = 1.00000
P(sent = 0    0
Name: sent, dtype: int64 | Sports) = 1.00000
P(sequence = 0    0
Name: sequence, dtype: int64 | Sports) = 1.

P(table = 0    0
Name: table, dtype: int64 | Sports) = 0.90000
P(taiwan = 0    0
Name: taiwan, dtype: int64 | Sports) = 1.00000
P(take = 0    0
Name: take, dtype: int64 | Sports) = 1.00000
P(taken = 0    0
Name: taken, dtype: int64 | Sports) = 1.00000
P(takes = 0    0
Name: takes, dtype: int64 | Sports) = 1.00000
P(taking = 0    0
Name: taking, dtype: int64 | Sports) = 0.90000
P(talent = 0    0
Name: talent, dtype: int64 | Sports) = 1.00000
P(talented = 0    0
Name: talented, dtype: int64 | Sports) = 1.00000
P(talents = 0    0
Name: talents, dtype: int64 | Sports) = 1.00000
P(talk = 0    0
Name: talk, dtype: int64 | Sports) = 1.00000
P(talking = 0    0
Name: talking, dtype: int64 | Sports) = 1.00000
P(talks = 0    0
Name: talks, dtype: int64 | Sports) = 0.90000
P(tam = 0    0
Name: tam, dtype: int64 | Sports) = 1.00000
P(tas = 0    0
Name: tas, dtype: int64 | Sports) = 0.90000
P(task = 0    0
Name: task, dtype: int64 | Sports) = 0.90000
P(taught = 0    0
Name: taught, dtype: int64 | Sp

Name: wall, dtype: int64 | Sports) = 0.90000
P(walls = 0    0
Name: walls, dtype: int64 | Sports) = 0.90000
P(wang = 0    0
Name: wang, dtype: int64 | Sports) = 1.00000
P(warm = 0    0
Name: warm, dtype: int64 | Sports) = 1.00000
P(warmup = 0    0
Name: warmup, dtype: int64 | Sports) = 0.90000
P(was = 0    0
Name: was, dtype: int64 | Sports) = 0.60000
P(water = 0    0
Name: water, dtype: int64 | Sports) = 0.80000
P(way = 0    0
Name: way, dtype: int64 | Sports) = 1.00000
P(ways = 0    0
Name: ways, dtype: int64 | Sports) = 1.00000
P(we = 0    0
Name: we, dtype: int64 | Sports) = 1.00000
P(weak = 0    0
Name: weak, dtype: int64 | Sports) = 1.00000
P(weakest = 0    0
Name: weakest, dtype: int64 | Sports) = 1.00000
P(weaknesses = 0    0
Name: weaknesses, dtype: int64 | Sports) = 0.90000
P(weave = 0    0
Name: weave, dtype: int64 | Sports) = 0.90000
P(weight = 0    0
Name: weight, dtype: int64 | Sports) = 0.90000
P(well = 0    0
Name: well, dtype: int64 | Sports) = 1.00000
P(went = 0    0


(1.8008087211030074e-72, 'Sports')

In [30]:
naive_bayes_classifier(bi_df.assign(category=y), "category", bi_test)


p(Education) = 0.50000
P(10 climb = 0    0
Name: 10 climb, dtype: int64 | Education) = 1.00000
P(10 however = 0    0
Name: 10 however, dtype: int64 | Education) = 0.90000
P(10 of = 0    0
Name: 10 of, dtype: int64 | Education) = 0.90000
P(100 sitting = 0    0
Name: 100 sitting, dtype: int64 | Education) = 0.90000
P(100m long = 0    0
Name: 100m long, dtype: int64 | Education) = 1.00000
P(100m run = 0    0
Name: 100m run, dtype: int64 | Education) = 1.00000
P(100m to = 0    0
Name: 100m to, dtype: int64 | Education) = 1.00000
P(118 campus = 0    0
Name: 118 campus, dtype: int64 | Education) = 0.90000
P(12 swing = 0    0
Name: 12 swing, dtype: int64 | Education) = 1.00000
P(120 practice = 0    0
Name: 120 practice, dtype: int64 | Education) = 0.90000
P(13 cross = 0    0
Name: 13 cross, dtype: int64 | Education) = 1.00000
P(1300 being = 0    0
Name: 1300 being, dtype: int64 | Education) = 1.00000
P(14 climb = 0    0
Name: 14 climb, dtype: int64 | Education) = 1.00000
P(15 climb = 0    0


P(activities based = 0    0
Name: activities based, dtype: int64 | Education) = 1.00000
P(activities by = 0    0
Name: activities by, dtype: int64 | Education) = 0.90000
P(activities claim = 0    0
Name: activities claim, dtype: int64 | Education) = 1.00000
P(activities football = 0    0
Name: activities football, dtype: int64 | Education) = 1.00000
P(activities from = 0    0
Name: activities from, dtype: int64 | Education) = 0.90000
P(activities share = 0    0
Name: activities share, dtype: int64 | Education) = 1.00000
P(activities varies = 0    0
Name: activities varies, dtype: int64 | Education) = 1.00000
P(activities without = 0    0
Name: activities without, dtype: int64 | Education) = 1.00000
P(activity aimed = 0    0
Name: activity aimed, dtype: int64 | Education) = 0.90000
P(activity directed = 0    0
Name: activity directed, dtype: int64 | Education) = 0.90000
P(activity engaged = 0    0
Name: activity engaged, dtype: int64 | Education) = 1.00000
P(activity for = 0    0
Name: 

Name: an increase, dtype: int64 | Education) = 1.00000
P(an independent = 0    0
Name: an independent, dtype: int64 | Education) = 0.90000
P(an individual = 0    0
Name: an individual, dtype: int64 | Education) = 1.00000
P(an inflatable = 0    0
Name: an inflatable, dtype: int64 | Education) = 1.00000
P(an influential = 0    0
Name: an influential, dtype: int64 | Education) = 0.90000
P(an innate = 0    0
Name: an innate, dtype: int64 | Education) = 0.90000
P(an institution = 0    0
Name: an institution, dtype: int64 | Education) = 0.90000
P(an interdisciplinary = 0    0
Name: an interdisciplinary, dtype: int64 | Education) = 0.90000
P(an international = 0    0
Name: an international, dtype: int64 | Education) = 1.00000
P(an introduction = 0    0
Name: an introduction, dtype: int64 | Education) = 1.00000
P(an object = 0    0
Name: an object, dtype: int64 | Education) = 0.90000
P(an objective = 0    0
Name: an objective, dtype: int64 | Education) = 1.00000
P(an obstacle = 0    0
Name: an

Name: and others, dtype: int64 | Education) = 0.90000
P(and out = 0    0
Name: and out, dtype: int64 | Education) = 1.00000
P(and over = 0    0
Name: and over, dtype: int64 | Education) = 1.00000
P(and parent = 0    0
Name: and parent, dtype: int64 | Education) = 0.90000
P(and parenting = 0    0
Name: and parenting, dtype: int64 | Education) = 0.90000
P(and pedagogy = 0    0
Name: and pedagogy, dtype: int64 | Education) = 0.90000
P(and performance = 0    0
Name: and performance, dtype: int64 | Education) = 0.90000
P(and personality = 0    0
Name: and personality, dtype: int64 | Education) = 0.90000
P(and physical = 0    0
Name: and physical, dtype: int64 | Education) = 0.90000
P(and popular = 0    0
Name: and popular, dtype: int64 | Education) = 0.90000
P(and potential = 0    0
Name: and potential, dtype: int64 | Education) = 0.90000
P(and practices = 0    0
Name: and practices, dtype: int64 | Education) = 0.90000
P(and practised = 0    0
Name: and practised, dtype: int64 | Education) 

P(are and = 0    0
Name: are and, dtype: int64 | Education) = 0.90000
P(are completed = 0    0
Name: are completed, dtype: int64 | Education) = 1.00000
P(are dedicated = 0    0
Name: are dedicated, dtype: int64 | Education) = 0.90000
P(are deep = 0    0
Name: are deep, dtype: int64 | Education) = 0.90000
P(are disagreements = 0    0
Name: are disagreements, dtype: int64 | Education) = 0.90000
P(are discarded = 0    0
Name: are discarded, dtype: int64 | Education) = 0.90000
P(are divided = 0    0
Name: are divided, dtype: int64 | Education) = 0.90000
P(are either = 0    0
Name: are either, dtype: int64 | Education) = 0.90000
P(are exclusive = 0    0
Name: are exclusive, dtype: int64 | Education) = 0.90000
P(are exemplified = 0    0
Name: are exemplified, dtype: int64 | Education) = 0.90000
P(are graded = 0    0
Name: are graded, dtype: int64 | Education) = 1.00000
P(are imparted = 0    0
Name: are imparted, dtype: int64 | Education) = 0.90000
P(are initially = 0    0
Name: are initially

...

Name: this perspective, dtype: int64 | Sports) = 1.00000
P(this phrase = 0    0
Name: this phrase, dtype: int64 | Sports) = 1.00000
P(this process = 0    0
Name: this process, dtype: int64 | Sports) = 1.00000
P(this program = 0    0
Name: this program, dtype: int64 | Sports) = 1.00000
P(this regard = 0    0
Name: this regard, dtype: int64 | Sports) = 1.00000
P(this respect = 0    0
Name: this respect, dtype: int64 | Sports) = 1.00000
P(this sense = 0    0
Name: this sense, dtype: int64 | Sports) = 1.00000
P(this shared = 0    0
Name: this shared, dtype: int64 | Sports) = 1.00000
P(this singaporean = 0    0
Name: this singaporean, dtype: int64 | Sports) = 0.90000
P(this theory = 0    0
Name: this theory, dtype: int64 | Sports) = 1.00000
P(this transmission = 0    0
Name: this transmission, dtype: int64 | Sports) = 1.00000
P(this view = 0    0
Name: this view, dtype: int64 | Sports) = 1.00000
P(this was = 0    0
Name: this was, dtype: int64 | Sports) = 1.00000
P(this way = 0    0
Name: t

Name: today over, dtype: int64 | Sports) = 1.00000
P(together at = 0    0
Name: together at, dtype: int64 | Sports) = 1.00000
P(together the = 0    0
Name: together the, dtype: int64 | Sports) = 0.90000
P(together they = 0    0
Name: together they, dtype: int64 | Sports) = 0.90000
P(tokyo hara = 0    0
Name: tokyo hara, dtype: int64 | Sports) = 1.00000
P(tolerance and = 0    0
Name: tolerance and, dtype: int64 | Sports) = 1.00000
P(took part = 0    0
Name: took part, dtype: int64 | Sports) = 1.00000
P(took the = 0    0
Name: took the, dtype: int64 | Sports) = 1.00000
P(tool has = 0    0
Name: tool has, dtype: int64 | Sports) = 1.00000
P(top 10 = 0    0
Name: top 10, dtype: int64 | Sports) = 1.00000
P(top flight = 0    0
Name: top flight, dtype: int64 | Sports) = 0.90000
P(top level = 0    0
Name: top level, dtype: int64 | Sports) = 0.90000
P(top marks = 0    0
Name: top marks, dtype: int64 | Sports) = 1.00000
P(top of = 0    0
Name: top of, dtype: int64 | Sports) = 0.90000
P(top progra

Name: values practices, dtype: int64 | Sports) = 1.00000
P(variables affected = 0    0
Name: variables affected, dtype: int64 | Sports) = 1.00000
P(variables that = 0    0
Name: variables that, dtype: int64 | Sports) = 1.00000
P(variables was = 0    0
Name: variables was, dtype: int64 | Sports) = 1.00000
P(variation between = 0    0
Name: variation between, dtype: int64 | Sports) = 1.00000
P(variations among = 0    0
Name: variations among, dtype: int64 | Sports) = 1.00000
P(varied significantly = 0    0
Name: varied significantly, dtype: int64 | Sports) = 1.00000
P(varies between = 0    0
Name: varies between, dtype: int64 | Sports) = 0.90000
P(variety of = 0    0
Name: variety of, dtype: int64 | Sports) = 1.00000
P(various attempts = 0    0
Name: various attempts, dtype: int64 | Sports) = 1.00000
P(various contexts = 0    0
Name: various contexts, dtype: int64 | Sports) = 1.00000
P(various counter = 0    0
Name: various counter, dtype: int64 | Sports) = 1.00000
P(various counterexamp

Name: while striving, dtype: int64 | Sports) = 1.00000
P(while subtext = 0    0
Name: while subtext, dtype: int64 | Sports) = 1.00000
P(while the = 0    0
Name: while the, dtype: int64 | Sports) = 1.00000
P(while there = 0    0
Name: while there, dtype: int64 | Sports) = 1.00000
P(while thick = 0    0
Name: while thick, dtype: int64 | Sports) = 1.00000
P(whilst gaisf = 0    0
Name: whilst gaisf, dtype: int64 | Sports) = 0.90000
P(whilst the = 0    0
Name: whilst the, dtype: int64 | Sports) = 0.90000
P(who are = 0    0
Name: who are, dtype: int64 | Sports) = 0.90000
P(who can = 0    0
Name: who can, dtype: int64 | Sports) = 1.00000
P(who expressed = 0    0
Name: who expressed, dtype: int64 | Sports) = 1.00000
P(who helps = 0    0
Name: who helps, dtype: int64 | Sports) = 1.00000
P(who positioned = 0    0
Name: who positioned, dtype: int64 | Sports) = 1.00000
P(who teaches = 0    0
Name: who teaches, dtype: int64 | Sports) = 1.00000
P(who were = 0    0
Name: who were, dtype: int64 | Spor

(1.0352296571629177e-115, 'Sports')

In [31]:
naive_bayes_classifier(tfidf_df.assign(category=y), "category", tfid_test)


p(Education) = 0.50000
P(100 = 0    0.0
Name: 100, dtype: float64 | Education) = 0.90000
P(100m = 0    0.0
Name: 100m, dtype: float64 | Education) = 1.00000
P(118 = 0    0.0
Name: 118, dtype: float64 | Education) = 0.90000
P(12 = 0    0.0
Name: 12, dtype: float64 | Education) = 1.00000
P(120 = 0    0.0
Name: 120, dtype: float64 | Education) = 0.90000
P(13 = 0    0.0
Name: 13, dtype: float64 | Education) = 1.00000
P(1300 = 0    0.0
Name: 1300, dtype: float64 | Education) = 1.00000
P(14 = 0    0.0
Name: 14, dtype: float64 | Education) = 1.00000
P(15 = 0    0.0
Name: 15, dtype: float64 | Education) = 0.90000
P(15th = 0    0.0
Name: 15th, dtype: float64 | Education) = 0.90000
P(16 = 0    0.0
Name: 16, dtype: float64 | Education) = 1.00000
P(165 = 0    0.0
Name: 165, dtype: float64 | Education) = 0.90000
P(1650s = 0    0.0
Name: 1650s, dtype: float64 | Education) = 0.90000
P(17th = 0    0.0
Name: 17th, dtype: float64 | Education) = 0.90000
P(18 = 0    0.0
Name: 18, dtype: float64 | Educati

Name: alexander, dtype: float64 | Education) = 0.90000
P(ali = 0    0.0
Name: ali, dtype: float64 | Education) = 1.00000
P(aligned = 0    0.0
Name: aligned, dtype: float64 | Education) = 0.90000
P(all = 0    0.0
Name: all, dtype: float64 | Education) = 0.30000
P(alleged = 0    0.0
Name: alleged, dtype: float64 | Education) = 0.90000
P(allow = 0    0.0
Name: allow, dtype: float64 | Education) = 0.90000
P(allowed = 0    0.0
Name: allowed, dtype: float64 | Education) = 1.00000
P(allowing = 0    0.0
Name: allowing, dtype: float64 | Education) = 1.00000
P(almost = 0    0.0
Name: almost, dtype: float64 | Education) = 1.00000
P(alone = 0    0.0
Name: alone, dtype: float64 | Education) = 0.80000
P(along = 0    0.0
Name: along, dtype: float64 | Education) = 0.90000
P(alongside = 0    0.0
Name: alongside, dtype: float64 | Education) = 0.90000
P(alpha = 0    0.0
Name: alpha, dtype: float64 | Education) = 0.90000
P(already = 0    0.0
Name: already, dtype: float64 | Education) = 0.80000
P(also = 0 

Name: benguela, dtype: float64 | Education) = 1.00000
P(berenguer = 0    0.0
Name: berenguer, dtype: float64 | Education) = 0.90000
P(beret = 0    0.0
Name: beret, dtype: float64 | Education) = 1.00000
P(besides = 0    0.0
Name: besides, dtype: float64 | Education) = 0.90000
P(best = 0    0.0
Name: best, dtype: float64 | Education) = 0.80000
P(better = 0    0.0
Name: better, dtype: float64 | Education) = 0.90000
P(betting = 0    0.0
Name: betting, dtype: float64 | Education) = 0.90000
P(between = 0    0.0
Name: between, dtype: float64 | Education) = 0.40000
P(beyond = 0    0.0
Name: beyond, dtype: float64 | Education) = 0.90000
P(bicycle = 0    0.0
Name: bicycle, dtype: float64 | Education) = 0.90000
P(bijutsu = 0    0.0
Name: bijutsu, dtype: float64 | Education) = 0.90000
P(billion = 0    0.0
Name: billion, dtype: float64 | Education) = 0.90000
P(biophilia = 0    0.0
Name: biophilia, dtype: float64 | Education) = 0.90000
P(birth = 0    0.0
Name: birth, dtype: float64 | Education) = 0.

Name: competent, dtype: float64 | Education) = 0.90000
P(competes = 0    0.0
Name: competes, dtype: float64 | Education) = 1.00000
P(competing = 0    0.0
Name: competing, dtype: float64 | Education) = 1.00000
P(competition = 0    0.0
Name: competition, dtype: float64 | Education) = 1.00000
P(competitions = 0    0.0
Name: competitions, dtype: float64 | Education) = 1.00000
P(competitive = 0    0.0
Name: competitive, dtype: float64 | Education) = 1.00000
P(complete = 0    0.0
Name: complete, dtype: float64 | Education) = 0.90000
P(completed = 0    0.0
Name: completed, dtype: float64 | Education) = 1.00000
P(completing = 0    0.0
Name: completing, dtype: float64 | Education) = 1.00000
P(complex = 0    0.0
Name: complex, dtype: float64 | Education) = 0.80000
P(component = 0    0.0
Name: component, dtype: float64 | Education) = 0.90000
P(components = 0    0.0
Name: components, dtype: float64 | Education) = 0.90000
P(comprehension = 0    0.0
Name: comprehension, dtype: float64 | Education) =

Name: designed, dtype: float64 | Education) = 0.90000
P(designer = 0    0.0
Name: designer, dtype: float64 | Education) = 0.90000
P(designing = 0    0.0
Name: designing, dtype: float64 | Education) = 0.90000
P(desirable = 0    0.0
Name: desirable, dtype: float64 | Education) = 0.90000
P(desport = 0    0.0
Name: desport, dtype: float64 | Education) = 1.00000
P(details = 0    0.0
Name: details, dtype: float64 | Education) = 0.90000
P(determination = 0    0.0
Name: determination, dtype: float64 | Education) = 1.00000
P(determine = 0    0.0
Name: determine, dtype: float64 | Education) = 0.80000
P(determined = 0    0.0
Name: determined, dtype: float64 | Education) = 1.00000
P(determining = 0    0.0
Name: determining, dtype: float64 | Education) = 1.00000
P(detrimental = 0    0.0
Name: detrimental, dtype: float64 | Education) = 0.90000
P(detritus = 0    0.0
Name: detritus, dtype: float64 | Education) = 0.90000
P(develop = 0    0.0
Name: develop, dtype: float64 | Education) = 1.00000
P(develo

...

P(rational = 0    0.0
Name: rational, dtype: float64 | Sports) = 1.00000
P(rationality = 0    0.0
Name: rationality, dtype: float64 | Sports) = 1.00000
P(rationally = 0    0.0
Name: rationally, dtype: float64 | Sports) = 1.00000
P(reaching = 0    0.0
Name: reaching, dtype: float64 | Sports) = 0.90000
P(reactivity = 0    0.0
Name: reactivity, dtype: float64 | Sports) = 1.00000
P(reading = 0    0.0
Name: reading, dtype: float64 | Sports) = 1.00000
P(realize = 0    0.0
Name: realize, dtype: float64 | Sports) = 1.00000
P(realizing = 0    0.0
Name: realizing, dtype: float64 | Sports) = 1.00000
P(really = 0    0.0
Name: really, dtype: float64 | Sports) = 1.00000
P(reasoning = 0    0.0
Name: reasoning, dtype: float64 | Sports) = 1.00000
P(reasons = 0    0.0
Name: reasons, dtype: float64 | Sports) = 1.00000
P(received = 0    0.0
Name: received, dtype: float64 | Sports) = 1.00000
P(receiver = 0    0.0
Name: receiver, dtype: float64 | Sports) = 1.00000
P(recent = 0    0.0
Name: recent, dtype: fl

Name: significantly, dtype: float64 | Sports) = 1.00000
P(similar = 0    0.0
Name: similar, dtype: float64 | Sports) = 0.80000
P(simply = 0    0.0
Name: simply, dtype: float64 | Sports) = 0.90000
P(simulate = 0    0.0
Name: simulate, dtype: float64 | Sports) = 1.00000
P(simulates = 0    0.0
Name: simulates, dtype: float64 | Sports) = 0.90000
P(simultaneous = 0    0.0
Name: simultaneous, dtype: float64 | Sports) = 0.90000
P(simultaneously = 0    0.0
Name: simultaneously, dtype: float64 | Sports) = 0.90000
P(since = 0    0.0
Name: since, dtype: float64 | Sports) = 0.90000
P(singapore = 0    0.0
Name: singapore, dtype: float64 | Sports) = 0.90000
P(singaporean = 0    0.0
Name: singaporean, dtype: float64 | Sports) = 0.90000
P(single = 0    0.0
Name: single, dtype: float64 | Sports) = 0.90000
P(singular = 0    0.0
Name: singular, dtype: float64 | Sports) = 0.90000
P(sitting = 0    0.0
Name: sitting, dtype: float64 | Sports) = 1.00000
P(situation = 0    0.0
Name: situation, dtype: float64 |

P(techniques = 0    0.0
Name: techniques, dtype: float64 | Sports) = 0.90000
P(technology = 0    0.0
Name: technology, dtype: float64 | Sports) = 1.00000
P(television = 0    0.0
Name: television, dtype: float64 | Sports) = 1.00000
P(temporary = 0    0.0
Name: temporary, dtype: float64 | Sports) = 1.00000
P(ten = 0    0.0
Name: ten, dtype: float64 | Sports) = 1.00000
P(tended = 0    0.0
Name: tended, dtype: float64 | Sports) = 1.00000
P(tendency = 0    0.0
Name: tendency, dtype: float64 | Sports) = 1.00000
P(tennis = 0    0.0
Name: tennis, dtype: float64 | Sports) = 0.90000
P(tenure = 0    0.0
Name: tenure, dtype: float64 | Sports) = 0.90000
P(term = 0    0.0
Name: term, dtype: float64 | Sports) = 0.70000
P(terms = 0    0.0
Name: terms, dtype: float64 | Sports) = 0.90000
P(terrain = 0    0.0
Name: terrain, dtype: float64 | Sports) = 0.90000
P(territories = 0    0.0
Name: territories, dtype: float64 | Sports) = 1.00000
P(tertiary = 0    0.0
Name: tertiary, dtype: float64 | Sports) = 1.00

Name: weight, dtype: float64 | Sports) = 0.90000
P(well = 0    0.0
Name: well, dtype: float64 | Sports) = 1.00000
P(went = 0    0.0
Name: went, dtype: float64 | Sports) = 1.00000
P(were = 0    0.0
Name: were, dtype: float64 | Sports) = 0.90000
P(western = 0    0.0
Name: western, dtype: float64 | Sports) = 1.00000
P(what = 0    0.0
Name: what, dtype: float64 | Sports) = 0.90000
P(when = 0    0.0
Name: when, dtype: float64 | Sports) = 0.80000
P(where = 0    0.0
Name: where, dtype: float64 | Sports) = 0.70000
P(whereas = 0    0.0
Name: whereas, dtype: float64 | Sports) = 1.00000
P(whether = 0    0.0
Name: whether, dtype: float64 | Sports) = 1.00000
P(which = 0    0.0
Name: which, dtype: float64 | Sports) = 0.60000
P(while = 0    0.0
Name: while, dtype: float64 | Sports) = 0.80000
P(whilst = 0    0.0
Name: whilst, dtype: float64 | Sports) = 0.80000
P(who = 0    0.0
Name: who, dtype: float64 | Sports) = 0.90000
P(whole = 0    0.0
Name: whole, dtype: float64 | Sports) = 1.00000
P(wide = 0   

(3.407930056453156e-74, 'Sports')